In [ ]:
import netCDF4
import numpy as np
import tiledb
from tiledb.cf import Group, GroupSchema
from tiledb.cf.engines.netcdf4_engine import NetCDF4ConverterEngine
import matplotlib.pyplot as plt

In [ ]:
# Create an example NetCDF file for the conversion.`
netcdf_file = "output/simple1.nc"
vfs = tiledb.VFS()
if not vfs.is_file(netcdf_file):
    # Create example numpy dataset
    x_data = np.linspace(-5.0, 5.0, 100)
    y_data = np.linspace(-5.0, 5.0, 100)
    xv, yv = np.meshgrid(x_data, y_data, sparse=True)
    A1_data = xv + yv
    A2_data = np.sin((xv / 2.0) ** 2 + yv ** 2)
    with netCDF4.Dataset(netcdf_file, mode="w") as dataset:
        dataset.setncatts({"title": "Simple dataset for examples"})
        dataset.createDimension("x", 100)
        dataset.createDimension("y", 100)
        A1 = dataset.createVariable("A1", np.float64, ("x", "y"))
        A1.setncattr("full_name", "Example matrix A1")
        A1.setncattr("description", "x + y")
        A1[:, :] = A1_data
        A2 = dataset.createVariable("A2", np.float64, ("x", "y"))
        A2[:, :] = A2_data
        A2.setncattr("full_name", "Example matrix A2")
        A2.setncattr("description", "sin((x/2)^2 + y^2")
        x1 = dataset.createVariable("x", np.float64, ("x",))
        x1[:] = x_data
        y = dataset.createVariable("y", np.float64, ("y",))
        y[:] = y_data
    print(f"Created example NetCDF file `{netcdf_file}`.")
else:
    print(f"Example NetCDF file `{netcdf_file}` already exists.")


In [ ]:
converter = NetCDF4ConverterEngine.from_file(
    netcdf_file,
    dim_dtype = np.uint32,
    tiles = {("x", "y"): (20,20), ("x",): (20,), ("y",): (20,)},
)
converter 

In [ ]:
# Rename array names to be more descriptive
converter.rename_array('array0', 'x')
converter.rename_array('array1', 'matrices')
converter.rename_array('array2', 'y')
# Run conversion
group_uri = "output/tiledb_simple1"
converter.convert(group_uri)

In [ ]:
group_schema = GroupSchema.load(group_uri)
group_schema

In [ ]:
# Example open attribute from generated TileDB group
with Group(group_uri, attr="x.data") as group:
    x = group.array[:]
with Group(group_uri, attr="y.data") as group:
    y = group.array[:]
with Group(group_uri, array="matrices") as group:
    data = group.array[...]
    A1 = data["A1"]
    A2 = data["A2"]
    a1_description = group.get_attr_metadata("A1")["description"]
    a2_description = group.get_attr_metadata("A2")["description"]


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
axes[0].contourf(x, y, A1);
axes[0].set_title(a1_description);
axes[1].contourf(x, y, A2);
axes[1].set_title(a2_description);